In [2]:
! pip install -r requirements.txt --quiet

# Connecting to a Remote MCP Server with Semantic Kernel

This notebook demonstrates how to connect to a remote MCP Server using **Semantic Kernel's** `MCPSsePlugin`. The **Model Context Protocol (MCP)** enables scalable and modular tool integration across distributed systems. 

<br/>

> **Why Use Model Context Protocol (MCP)?**
>
>MCP allows agents to discover, invoke, and manage tools dynamically across remote servers.  
>It promotes modularity, scalability, and separation of concerns, making it easier to maintain and extend AI systems as they grow in complexity.

In [7]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from dotenv import load_dotenv
from os import environ
import asyncio

from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

load_dotenv(override=True)

kernel = Kernel()

kernel.add_service(AzureChatCompletion(
    service_id="chat",
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    base_url=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] ))



## Connecting to a remote MCP server via sse

- Server-Sent Events (SSE) is a mechanism for establishing a continuous, one-way data stream from the server to a client, such as a web application. This allows the server to push updates or messages to the client without the client needing to constantly poll for new information

In [8]:
async with MCPStreamableHttpPlugin(
    name="sales",
    url=f"{environ['MCP_SERVER_URL']}",
) as sales_plugin:

    agent = ChatCompletionAgent(
        kernel=kernel, 
        name="SalesAgent", 
        plugins=[sales_plugin, ]
    )
    response = await agent.get_response(messages="Which products generated the most revenue?")
    print(response)

The products that generated the most revenue are:

1. **Brake Pad** (Category: Braking) - Total revenue: $275,750.00
2. **Oil Filter** (Category: Engine) - Total revenue: $83,865.00
3. **Spark Plug** (Category: Electrical) - Total revenue: $44,112.00


In [9]:
from contextlib import AsyncExitStack

plugin_names = ["weather", "search", "energy"]

messages = [
    "What data centers are in 'critical'?",
    "Describe the resource intensity of data center facility infrastructure"
]


async with AsyncExitStack() as stack:
    plugins = []
    for plugin_name in plugin_names:
        plugin = await stack.enter_async_context(
            MCPSsePlugin(
                name=plugin_name,
                url=f"{environ['MCP_SERVER_URL']}/{plugin_name}/sse"
            )
        )
        plugins.append(plugin)

    agent = ChatCompletionAgent(
        kernel=kernel,
        name="MultiPluginAgent",
        plugins=plugins
    )


    for msg in messages:
        print("------------------------------------------------\n")
        print(msg)
        response = await agent.get_response(messages=msg)
        print(response)
        print("\n------------------------------------------------\n")


NameError: name 'MCPSsePlugin' is not defined